# GenVeg Main Script
GenVeg is a generalized vegetation growth model that simulates growth due to
photosynthesis as well as processes that control vegetation distribution such
as mortality, senescence, dispersal, and colonization. The model utilizes a 
mixture of existing and new landlab components to setup the model, simulates
relevant vegetation community dynamics with the local environment, and analyzes
results of the simulation. 

## Main model control set up
### Import components

In [ ]:
import os
import tempfile
from datetime import date
from pathlib import Path

import numpy as np
import pandas as pd
from matplotlib.pyplot import figure, plot, show, subplot, title, xlabel, ylabel

from landlab import RasterModelGrid, imshow_grid, load_params
from landlab.components.genveg import GenVeg, PlantGrowth, VegParams
from landlab.data_record import DataRecord
from landlab.io.netcdf import write_netcdf

### Define time controls

In [ ]:
sim_start_date = np.datetime64("2010-01-01")
tot_sim_len_y = 30
veg_time_step_d = 1
env_time_step_d = 1
dt = np.timedelta64(min([veg_time_step_d, env_time_step_d]), "D")
run_length_d = 365

### Define filenames for input files
##### *Note should be located in same folder as this .ipynb

In [ ]:
VegInputFile = "GenVeg_params_inputs_1col.xlsx"

### Read in parameter and data input files
Identify model processes to be included so all required parameters are defined and calculated

In [ ]:
inp_file = Path(VegInputFile).absolute()
VegParams(
    fpath=inp_file, processes=["plantsize", "dispersal", "colonization", "storage"]
)
# VegParams(processes=['plantsize','dispersal','colonization','storage','mortality'])
vegparams = load_params("veg_params.yml")
print(vegparams)

### Set up model grid using landlab RasterModelGrid functions

- Create the model grid
- Initialize live vegetation biomass and plant functional type
- Initialize the air temperature driver across the grid

##### **Note variable names should be consistent with landlab standards

In [ ]:
# Create grid 5x5 node grid with 2 m node spacing and xy of origin in lat long roughly corresponding to Island Beach State Park
pg = RasterModelGrid((6, 6), 2.0, xy_of_reference=(-74.08, 39.79))
pg.axis_units = ("m", "m")


# Initialize with a random mean air temperture
_ = pg.add_field(
    "air__temperature_C",
    np.random.uniform(low=12, high=14, size=pg.number_of_cells),
    at="cell",
    units="C",
)
_ = pg.add_field(
    "radiation__net_flux",
    np.random.uniform(low=0, high=180, size=pg.number_of_cells),
    at="cell",
    units="W/m^2",
)

Now we initialize plants. We can do this in multiple ways. Here we are going to seed two species randomly into grid cells and then count how many plants are in each cell to initialize the new fields.

In [ ]:
# Initialize plant functional type as burial tolerant stabilizer or dune builder and randomly place on grid
plant_species_init = ["BTS", "DB"]
plants_on_grid = []
for ind, species in enumerate(plant_species_init):
    new_plants = np.random.choice(["null", species], pg.number_of_cells)
    if ind == 0:
        plants_on_grid = new_plants
    else:
        plants_on_grid = np.stack((plants_on_grid, new_plants), axis=1)

_ = pg.add_field("vegetation__plant_species", plants_on_grid, at="cell", clobber=True)

# occupied_ids=np.where(pg.at_cell['vegetation__plant_species']=='BTS')

pidval=0
plants=pd.DataFrame(columns=['pid','species','cell_index'])
for cell in range(pg.number_of_cells):
    cell_index=cell
    cell_plants=pg['cell']['vegetation__plant_species'][cell]
    for plant in cell_plants:
        species=plant
        newrow=[pidval,species,cell_index]
        plants.loc[pidval]=newrow
        pidval += 1

pid=plants.pid.loc[plants['species']=='BTS'].to_numpy(int)
total_biomass=np.random.rand(pid.shape[0])
total_biomass2=np.random.rand(6)

species_set=np.vstack((pid,total_biomass))
species_set2=np.vstack((total_biomass2,total_biomass2))
species_set=np.transpose(species_set)
species_set2=np.transpose(species_set2)
biomass=np.concatenate((species_set,species_set2), axis=0)
print(biomass)
biomass=np.transpose(biomass)
print(biomass)

In [ ]:
# This was a check to see if I could save a list as a field and it is possible so we can have multiple species per cell
# veg_biomass=np.where(pg.at_cell['vegetation__plant_species'] != 'null',1,0)*np.random.uniform(low=0, high=200,size=pg.number_of_cells)
# veg_biomass_part=veg_biomass/3
# veg_biomass_tot=[]
# for ind, item in enumerate(veg_biomass):
#    veg_biomass_tot.append([item, veg_biomass_part[ind],veg_biomass_part[ind],veg_biomass_part[ind]])
# print(veg_biomass_tot)

### Plot grid with plant distribution and initial biomass

Use RasterModelGrid plot functions to visualize initial model grid

#### Vegetation type 
- 0=Bare
- 1=Annual
- 2=Herbaceous Perennial 
- 3=Woody Perennial

In [ ]:
# Need to pudate to use colormap based on unique strings
# imshow_grid(pg, 'vegetation__plant_species', at='cell',colorbar_label='Vegetation Type')
# show()

In [ ]:
# imshow_grid(pg, 'vegetation__live_biomass', at='cell', colorbar_label='Live biomass g')
# show()

### Define model time variables
Determine basic model timestep and counter for model loop

sim_end = date(sim_start_date.year+tot_sim_len_y,sim_start_date.month,sim_start_date.day)
dt = min([veg_time_step_d,env_time_step_d])
#data['dt']=dt
day_freq = str(dt)+'D'
date_counter = pd.date_range(start=sim_start_date,end=sim_end,freq=day_freq).strftime('%Y-%m-%d').tolist()
step_counter=range(len(date_counter))
model_counter=tuple(zip(step_counter,date_counter))

Initialize crops PlantGrowth class with veg_params from Excel file

In [ ]:
croprun = GenVeg(pg, dt, sim_start_date, vegparams)
print(croprun.plant_ID())

## Main model loop script
This is the main model loop that controls the simulation that
- Advances model time 
- Updates environmental variables
- Simulates changes to vegetation communities
- Stores outputs

In [ ]:
# This runs through fine for day 1 of growing season. We need to decide how to track current day
for i in range(run_length_d):
    # update temp attribute on grid

    # update solar radiation on grid
    croprun.run_one_step(dt)
    print(croprun.inc_growth())
    # print(i)
    # if this is a save timestep, save grid object to netcdf
    # create record method in croprun that uses datarecord to save PlantGrowth object and grid at time
    # if i in savetime:
    #    croprun.record()
    # to view output:
    # croprun._record.dataset (for grid)
    # croprun._record_df (for plant dataframe)
    croprun.save_output()

In [ ]:
temp_dir = tempfile.mkdtemp()
os.chdir(temp_dir)
write_netcdf("test.nc", pg, format="NETCDF3_64BIT", time=step_counter[0])